In [43]:
# Importing required packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the iris data in dataset variable
dataset = load_iris()
dataset.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [3]:
# Creating a complete data frame on which the model is going to be created
# Store it as df
df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
df['target'] = dataset.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
# Separating the input and output variables
X = df.drop(['target'], axis = 1)
y = df.target

In [14]:
# Train test split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [16]:
# Build a SVM classifier model, train, predict, evaluation
svm_model = SVC(kernel = 'rbf', C = 30, gamma = 'auto')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of the model is:', accuracy )
print('The model score is:', svm_model.score(X_test, y_test))

Accuracy of the model is: 0.9666666666666667
The model score is: 0.9666666666666667


In [19]:
# Getting the Cross Validation Score for the SVM model with linear kernel, C is 10 and gamma is auto
cross_val_score(SVC(kernel = 'linear', C = 10, gamma = 'auto'),X, y, cv = 5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [21]:
# Getting the Cross Validation Score for the SVM model with rbf kernel, C is 20 and gamma is auto
cross_val_score(SVC(kernel = 'rbf', C = 10, gamma = 'auto'),X, y, cv = 5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [22]:
# Getting the Cross Validation Score for the SVM model with rbf kernel, C is 20 and gamma is auto
cross_val_score(SVC(kernel = 'rbf', C = 20, gamma = 'auto'),X, y, cv = 5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [25]:
# Writing same codes repeaedly is not possible
# We can use for loop for this purpose
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}

for kval in kernels:
  for cval in C:
    cv_scores = cross_val_score(SVC(kernel = kval , C = cval, gamma = 'auto'),X, y, cv = 5)
    avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

# Get the Average Scores
avg_scores

# This method also has some limitations
# If there are four variables like kernels and C, then four loops to write and number of iteration increases
# So, we use sklearn's GridSearchCV to do that

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [29]:
# Use Sklearn's GridSearchCV for hyper-parameter tuning purpose
# Name the classifier model as clf
clf = GridSearchCV(SVC(gamma = 'auto'),{'C':[1,10,20], 'kernel':['linear', 'rbf']}, cv = 5, return_train_score = False)

clf.fit(dataset.data, dataset.target)
clf.cv_results_

{'mean_fit_time': array([0.00368648, 0.00586038, 0.00569582, 0.00584311, 0.00498753,
        0.00593982]),
 'std_fit_time': array([0.00179707, 0.00505598, 0.00335866, 0.00533793, 0.00602866,
        0.00324286]),
 'mean_score_time': array([0.00460014, 0.00289102, 0.00120645, 0.00136209, 0.00291343,
        0.00247288]),
 'std_score_time': array([0.00419297, 0.00287601, 0.00010908, 0.0003029 , 0.00212181,
        0.00218456]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'}],
 'split0_test_score': ar

In [30]:
# Converting the cv results in a pandas data frame
cv_result =pd.DataFrame(clf.cv_results_)
cv_result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003686,0.001797,0.004600,0.004193,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.005860,0.005056,0.002891,0.002876,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.005696,0.003359,0.001206,0.000109,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
3,0.005843,0.005338,0.001362,0.000303,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.004988,0.006029,0.002913,0.002122,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6
5,0.005940,0.003243,0.002473,0.002185,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5


In [31]:
# All column of cv_result is not needed
# So, store the needed columns in a main_result variable
main_result = cv_result[['param_C', 'param_kernel', 'mean_test_score']]
main_result

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,1,rbf,0.980000
2,10,linear,0.973333
3,10,rbf,0.980000
4,20,linear,0.966667
5,20,rbf,0.966667


In [32]:
# What are the other parameters of the clf model
dir(clf)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run

In [33]:
# Check other parameters of SVM model
dir(svm_model)

['C',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_proba',
 '_compute_kernel',
 '_decision_function',
 '_dense_decision_function',
 '_dense_fit',
 '_dense_predict',
 '_dense_predict_proba',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_dual_coef_',
 '_estimator_type',
 '_gamma',
 '_get_coef',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_impl',
 '_intercept_',
 '_more_tags

In [34]:
# Get the best score of the clf model
clf.best_score_

0.9800000000000001

In [35]:
# Which parameters yields the best scores
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [38]:
# Use RandomizedSearchCV object to build the classification model using SVM
rs = RandomizedSearchCV(SVC(gamma = 'auto'), {'C':[1,10,20], 'kernel':['linear', 'rbf']}, cv = 5, return_train_score = False, n_iter = 2)
rs.fit(dataset.data, dataset.target) # It will go through only 2 iteration
rs.cv_results_

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'mean_fit_time': array([0.00151329, 0.00121617]),
 'std_fit_time': array([6.34909946e-04, 9.04964820e-05]),
 'mean_score_time': array([0.00090022, 0.00078135]),
 'std_score_time': array([1.44012511e-04, 3.91071593e-05]),
 'param_kernel': masked_array(data=['rbf', 'rbf'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[1, 10],
              mask=[False, False],
        fill_value=999999),
 'params': [{'kernel': 'rbf', 'C': 1}, {'kernel': 'rbf', 'C': 10}],
 'split0_test_score': array([0.96666667, 0.96666667]),
 'split1_test_score': array([1., 1.]),
 'split2_test_score': array([0.96666667, 0.96666667]),
 'split3_test_score': array([0.96666667, 0.96666667]),
 'split4_test_score': array([1., 1.]),
 'mean_test_score': array([0.98, 0.98]),
 'std_test_score': array([0.01632993, 0.01632993]),
 'rank_test_score': array([1, 1], dtype=int32)}

In [39]:
# String the result
rs_result = pd.DataFrame(rs.cv_results_)
rs_result = rs_result[['param_C', 'param_kernel', 'mean_test_score']]
rs_result

,param_C,param_kernel,mean_test_score
0,1,rbf,0.98
1,10,rbf,0.98


#### Now comes the most interesting part
#### How to choose the best model

In [41]:
# Define the model object and its parameters
model_params = {
    'svm': {
        'model': SVC(gamma = 'auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver = 'liblinear',multi_class = 'auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [44]:
# Write a for loop for each model
# Using GridSearchCV
# Store the scores
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv = 5, return_train_score = False)
    clf.fit(dataset.data, dataset.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
